<a href="https://colab.research.google.com/github/MelikeBadem/Forecast-WeatherML/blob/main/forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import re
import numpy as np

# Tarih aralığı
start_date = datetime(2020, 12, 1)
end_date = datetime(2025, 1, 20)

# Boş bir liste veri için
all_data = []

# Her tarih için döngü
date = start_date
while date <= end_date:
    formatted_date = date.strftime("%Y-%m-%d")
    url = f"https://forecast.weather.gov/MapClick.php?lat=37.7749&lon=-122.4194&date={formatted_date}"

    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, "html.parser")

    # Ayrıntılı tahmin kısmını bul
    detailed_forecast = soup.find("div", {"id": "detailed-forecast-body"})
    if detailed_forecast:
        forecast_items = detailed_forecast.find_all("div", class_="row-forecast")

        for item in forecast_items:
            period = item.find("div", class_="forecast-label").text.strip()  # Zaman dilimi
            desc = item.find("div", class_="forecast-text").text.strip()  # Hava durumu

            # Hava durumu özeti ayıklama
            weather_summary_match = re.match(r"^(.*?)[.,]", desc)
            weather_summary = weather_summary_match.group(1) if weather_summary_match else "Bilinmiyor"

            # Rüzgar hızı regex ile çekiliyor
            wind_match = re.search(r"wind.*?(\d+\s*mph)", desc, re.IGNORECASE)
            wind_speed = wind_match.group(1) if wind_match else "Bilinmiyor"

            # Sıcaklık regex ile çekiliyor
            temp_match = re.search(r"high near (\d+)|low around (\d+)", desc, re.IGNORECASE)
            temp = temp_match.group(1) if temp_match and temp_match.group(1) else (temp_match.group(2) if temp_match else "Bilinmiyor")

            # Veriyi ekle
            all_data.append((formatted_date, period, weather_summary, temp, wind_speed))

    # Bir sonraki güne geç
    date += timedelta(days=1)

# DataFrame'e dönüştür
df = pd.DataFrame(all_data, columns=["Tarih", "Zaman Dilimi", "Hava Durumu Özeti", "Sıcaklık", "Rüzgar Hızı"])
print(df)

# CSV'ye kaydet
df.to_csv("gecmis_hava_durumu.csv", index=False)

            Tarih     Zaman Dilimi      Hava Durumu Özeti Sıcaklık Rüzgar Hızı
0      2020-12-01            Today  Patchy fog before 9am       58       5 mph
1      2020-12-01          Tonight   Patchy fog after 2am       43  Bilinmiyor
2      2020-12-01           Sunday  Patchy fog before 9am       56  Bilinmiyor
3      2020-12-01     Sunday Night           Mostly clear       44       5 mph
4      2020-12-01     M.L.King Day                  Sunny       61       9 mph
...           ...              ...                    ...      ...         ...
19651  2025-01-20        Wednesday                  Sunny       61  Bilinmiyor
19652  2025-01-20  Wednesday Night           Mostly clear       46  Bilinmiyor
19653  2025-01-20         Thursday                  Sunny       64  Bilinmiyor
19654  2025-01-20   Thursday Night           Mostly clear       46  Bilinmiyor
19655  2025-01-20           Friday                  Sunny       62  Bilinmiyor

[19656 rows x 5 columns]


In [37]:


# 1. CSV dosyasını yükle
dosya_adi = "gecmis_hava_durumu.csv"
df = pd.read_csv(dosya_adi)

# 2. Tarih sütununu datetime formatına çevir
df['Tarih'] = pd.to_datetime(df['Tarih'])

# 3. Rüzgar Hızı sütununu temizle ve sayısal formata çevir
def temizle_ve_cevir(rüzgar_hizi):
    if isinstance(rüzgar_hizi, str) and "mph" in rüzgar_hizi:
        return float(rüzgar_hizi.replace(" mph", ""))
    elif rüzgar_hizi == "Bilinmiyor":
        return np.nan
    else:
        return rüzgar_hizi

df['Rüzgar Hızı'] = df['Rüzgar Hızı'].apply(temizle_ve_cevir)

# 4. Aynı tarihteki ortalamaları hesaplayarak eksik değerleri doldur
ortalama_degerler = df.groupby('Tarih')['Rüzgar Hızı'].transform('mean')
df['Rüzgar Hızı'] = df['Rüzgar Hızı'].fillna(ortalama_degerler)

# 5. Sonucu yeni bir CSV dosyasına kaydet
df.to_csv("guncellenmis_hava_durumu.csv", index=False)

print("İşlem tamamlandı! Güncellenmiş veri 'guncellenmis_hava_durumu.csv' dosyasına kaydedildi.")
print(df.head(5))

İşlem tamamlandı! Güncellenmiş veri 'guncellenmis_hava_durumu.csv' dosyasına kaydedildi.
       Tarih  Zaman Dilimi      Hava Durumu Özeti  Sıcaklık  Rüzgar Hızı
0 2020-12-01         Today  Patchy fog before 9am        58     5.000000
1 2020-12-01       Tonight   Patchy fog after 2am        43     6.333333
2 2020-12-01        Sunday  Patchy fog before 9am        56     6.333333
3 2020-12-01  Sunday Night           Mostly clear        44     5.000000
4 2020-12-01  M.L.King Day                  Sunny        61     9.000000


In [38]:
import pandas as pd

# CSV dosyasını okuma
data = pd.read_csv("guncellenmis_hava_durumu.csv")
# DataFrame'e dönüştürme
df = pd.DataFrame(data)

df['Hava Durumu Özeti'] = df['Hava Durumu Özeti'].str.replace(r"after \d+am|before \d+am", "", regex=True).str.strip()

# Hava durumu kategorilerini belirleme
hava_durumu_kategorileri = ["Sunny", "Partly sunny", "Partly cloudy", "Clear", "Mostly clear", "Patchy fog"]

# One-Hot Encoding işlemi
for kategori in hava_durumu_kategorileri:
    df[kategori] = df['Hava Durumu Özeti'].apply(lambda x: 1 if kategori in x else 0)

# Sonuçları CSV dosyasına kaydetme
df.to_csv("guncellenmis_hava_durumu.csv", index=False)

# İlk 5 satırı gösterme
print(df.head())

        Tarih  Zaman Dilimi Hava Durumu Özeti  Sıcaklık  Rüzgar Hızı  Sunny  \
0  2020-12-01         Today        Patchy fog        58     5.000000      0   
1  2020-12-01       Tonight        Patchy fog        43     6.333333      0   
2  2020-12-01        Sunday        Patchy fog        56     6.333333      0   
3  2020-12-01  Sunday Night      Mostly clear        44     5.000000      0   
4  2020-12-01  M.L.King Day             Sunny        61     9.000000      1   

   Partly sunny  Partly cloudy  Clear  Mostly clear  Patchy fog  
0             0              0      0             0           1  
1             0              0      0             0           1  
2             0              0      0             0           1  
3             0              0      0             1           0  
4             0              0      0             0           0  


In [39]:
# Zaman Dilimi sütununa göre "Night" ve "Non-Night" sütunları ekliyoruz
df['Night'] = df['Zaman Dilimi'].apply(lambda x: 1 if 'Night' in x else 0)
df['Non-Night'] = df['Zaman Dilimi'].apply(lambda x: 1 if 'Night' not in x else 0)
df.to_csv("guncellenmis_hava_durumu.csv", index=False)
print(df.head(5))


        Tarih  Zaman Dilimi Hava Durumu Özeti  Sıcaklık  Rüzgar Hızı  Sunny  \
0  2020-12-01         Today        Patchy fog        58     5.000000      0   
1  2020-12-01       Tonight        Patchy fog        43     6.333333      0   
2  2020-12-01        Sunday        Patchy fog        56     6.333333      0   
3  2020-12-01  Sunday Night      Mostly clear        44     5.000000      0   
4  2020-12-01  M.L.King Day             Sunny        61     9.000000      1   

   Partly sunny  Partly cloudy  Clear  Mostly clear  Patchy fog  Night  \
0             0              0      0             0           1      0   
1             0              0      0             0           1      0   
2             0              0      0             0           1      0   
3             0              0      0             1           0      1   
4             0              0      0             0           0      0   

   Non-Night  
0          1  
1          1  
2          1  
3          0  
4    

In [40]:
import pandas as pd

# Mevsimlere göre tarihlere atama yapalım
def mevsim_belirle(tarih):
    if pd.to_datetime(f"{tarih.year}-06-13") <= tarih <= pd.to_datetime(f"{tarih.year}-10-24"):
        return "Yaz"
    elif pd.to_datetime(f"{tarih.year}-12-04") <= tarih <= pd.to_datetime(f"{tarih.year+1}-02-04"):
        return "Serin"
    else:
        return "Kış"

# Yeni sütunlar ekleyelim
df['Yaz'] = df['Tarih'].apply(lambda x: 1 if mevsim_belirle(pd.to_datetime(x)) == 'Yaz' else 0)
df['Serin'] = df['Tarih'].apply(lambda x: 1 if mevsim_belirle(pd.to_datetime(x)) == 'Serin' else 0)
df['Kış'] = df['Tarih'].apply(lambda x: 1 if mevsim_belirle(pd.to_datetime(x)) == 'Kış' else 0)

# Sonuçları kaydet
df.to_csv("guncellenmis_hava_durumu.csv", index=False)

# Sonuç
print(df)


            Tarih     Zaman Dilimi Hava Durumu Özeti  Sıcaklık  Rüzgar Hızı  \
0      2020-12-01            Today        Patchy fog        58     5.000000   
1      2020-12-01          Tonight        Patchy fog        43     6.333333   
2      2020-12-01           Sunday        Patchy fog        56     6.333333   
3      2020-12-01     Sunday Night      Mostly clear        44     5.000000   
4      2020-12-01     M.L.King Day             Sunny        61     9.000000   
...           ...              ...               ...       ...          ...   
19651  2025-01-20        Wednesday             Sunny        61     6.333333   
19652  2025-01-20  Wednesday Night      Mostly clear        46     6.333333   
19653  2025-01-20         Thursday             Sunny        64     6.333333   
19654  2025-01-20   Thursday Night      Mostly clear        46     6.333333   
19655  2025-01-20           Friday             Sunny        62     6.333333   

       Sunny  Partly sunny  Partly cloudy  Clear  M